In [ ]:
# Mount to Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Define Project Folder
FOLDERNAME = 'Colab\ Notebooks/Group_A_Project'

%cd drive/MyDrive/$FOLDERNAME

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Group_A_Project


In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
# Define device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
# Seed for same output
torch.manual_seed(42)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [ ]:
# Reading in our file
raw_data = pd.read_csv('Amazon_Unlocked_Mobile.csv')

In [ ]:
reviews = raw_data['Reviews']
reviews = reviews.astype(str)
labels = raw_data['Rating']
labels.replace({1:0, 2:0, 3:1, 4:2, 5:2}, inplace=True)

In [ ]:
patterns = ['<br />', '--', '.', ',', '!', '?', ')', '(', ';', ':', '*', '~', '_', "'", '"']
replacements = [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '', '']

In [ ]:
def preprocessing(reviews, patterns, replacements):
    lst = []
    for i in range(len(reviews)):
        review = reviews[i].lower()
        for pattern, replacement in zip(patterns, replacements):
            review = review.replace(pattern, replacement)
        lst.append(review)
    return lst

In [ ]:
reviews = preprocessing(reviews, patterns, replacements)

In [ ]:
print(reviews)
print(labels)

In [ ]:
cleaned_reviews = preprocessing(reviews, patterns, replacements)

# Tokenize the reviews and find the length of each tokenized review
tokenized_reviews = [review.split() for review in cleaned_reviews]
num_tokens_per_review = [len(tokens) for tokens in tokenized_reviews]

# Find the longest number of tokens in any review
# longest_num_tokens = max(num_tokens_per_review)
# print(longest_num_tokens)

In [ ]:
num_train = 244031
num_val = 104585
longest_num_tokens = 200

In [ ]:
def indexing_tokens():
    indices = {'<SOS>': 0, '<EOS>': 1, '<PAD>': 2, '<UNK>': 3}
    index = 4
    for i in range(num_train):
        review = reviews[i].split()
        for token in review:
            if token not in indices:
                indices[token] = index
                index += 1
    return indices

In [ ]:
def get_data(indices, longest_line_tokens, mode='train'):
    data = []
    Y = []
    if mode == 'train':
      for i in range(num_train):
        one_train_data = []
        y, review = labels[i], reviews[i]
        tokens = review.split()
        for token in tokens:
          one_train_data.append(indices[token])
          if len(one_train_data) == longest_line_tokens:
            break
        while len(one_train_data) < longest_line_tokens:
          one_train_data.append(indices['<PAD>'])
        one_train_data.insert(0, indices['<SOS>'])
        one_train_data.append(indices['<EOS>'])
        data.append(one_train_data)
        Y.append(y)
    else:
      for i in range(num_train, num_train+num_val):
        one_val_data = []
        y, review = labels[i], reviews[i]
        tokens = review.split()
        for token in tokens:
          if token in indices:
            one_val_data.append(indices[token])
          else:
            one_val_data.append(indices['<UNK>'])
          if len(one_val_data) == longest_line_tokens:
            break
        while len(one_val_data) < longest_line_tokens:
          one_val_data.append(indices['<PAD>'])
        one_val_data.insert(0, indices['<SOS>'])
        one_val_data.append(indices['<EOS>'])
        data.append(one_val_data)
        Y.append(y)
    return data, Y

In [ ]:
# Loading Training Data & Val Data
indices = indexing_tokens()
training_data, training_labels = get_data(indices, longest_num_tokens)
val_data, val_labels = get_data(indices, longest_num_tokens, mode='val')

In [ ]:
print('Number of training:', len(training_data))
print('Number of validation:', len(val_data))
print('Length of corpus:', len(indices))

Number of training: 244031
Number of validation: 104585
Length of corpus: 56793


In [ ]:
# Create tensors of train & val
import numpy as np
train_tensor = torch.tensor(np.array(training_data))
train_labels_tensor = torch.tensor(np.array(training_labels))
val_tensor = torch.tensor(np.array(val_data))
val_labels_tensor = torch.tensor(np.array(val_labels))

In [ ]:
print('Train Tensor:', train_tensor.shape)
print('Val Tensor:', val_tensor.shape)

In [ ]:
vocab_size = 56793
embedding_dim = 300
hidden_dim = 256
sequence_len = 202
output_dim = 3
print_every = 400
batch_size = 64

In [ ]:
class MyModel(nn.Module):
  def __init__(self, corpus_len, embedding_dim, hidden_dim, output_dim):
    super().__init__()
    self.embedding = nn.Embedding(corpus_len, embedding_dim)
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, x):
    out = self.embedding(x)
    output, (h_n, c_n) = self.lstm(out)
    # out = self.fc(output[:, -1, :])
    out = self.fc(h_n).squeeze()
    return out

In [ ]:
model = MyModel(vocab_size, embedding_dim, hidden_dim, output_dim)
model = model.cuda()

In [ ]:
mini_trains = DataLoader(train_tensor, batch_size=batch_size)
mini_train_labels = DataLoader(training_labels, batch_size=batch_size)

mini_vals = DataLoader(val_tensor, batch_size=batch_size)
mini_val_labels = DataLoader(val_labels, batch_size=batch_size)

In [ ]:
iterator = iter(mini_trains)
print(next(iterator).shape)

iterator = iter(mini_train_labels)
print(next(iterator).shape)

torch.Size([64, 202])
torch.Size([64])


In [ ]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
# Training Procedure
def train(num_epoch, model, mini_trains, mini_train_labels, mini_vals, mini_val_labels, device, loss_function, optimizer):
    # 用於記錄損失和準確率的列表
    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []

    for epoch in range(num_epoch):
        num_iters = 0
        total_train_loss = 0
        correct_train_preds = 0
        total_train_samples = 0

        for x, y in zip(mini_trains, mini_train_labels):
            model.train()

            x = x.to(device)
            y = y.to(device)

            scores = model(x)
            loss = loss_function(scores, y)
            total_train_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            predictions = scores.argmax(dim=1)
            correct_train_preds += (predictions == y).sum().item()
            total_train_samples += y.size(0)

            if num_iters % print_every == 0:
                print(f"Iteration {num_iters}, Loss: {loss.item():.4f}")
                evaluate_predictor(model, epoch, mini_vals, mini_val_labels, device, loss_function)
            num_iters += 1

        # 計算每個 epoch 的訓練準確率和損失
        train_accuracy = correct_train_preds / total_train_samples
        avg_train_loss = total_train_loss / len(mini_trains)
        train_accuracies.append(train_accuracy)
        train_losses.append(avg_train_loss)

        print(f'Epoch [{epoch + 1}] Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}')

        # 驗證每個 epoch 的 val 準確率跟 loss
        val_accuracy, val_loss = evaluate_predictor(model, epoch, mini_vals, mini_val_labels, device, loss_function)
        val_accuracies.append(val_accuracy)
        val_losses.append(val_loss)

        print(f'Epoch [{epoch + 1}] Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

In [ ]:
# Evaluate Procedure
def evaluate_predictor(model, epoch, mini_vals, mini_val_labels, device, loss_function):
    model.eval()
    with torch.no_grad():
        acc_count = 0
        total_loss = 0.0
        total_samples = 0
        for x, y in zip(mini_vals, mini_val_labels):
            x = x.to(device)
            y = y.to(device)

            scores = model(x)
            predictions=scores.max(1)[1]

            # Calculate the loss for the current batch
            loss = loss_function(scores, y)
            total_loss += loss.item()

            # Update accuracy count
            acc = predictions.eq(y).sum().item()
            acc_count += acc
            total_samples += y.size(0)  # Track the number of samples in the batch

    accuracy = acc_count / total_samples
    avg_loss = total_loss / len(mini_vals)

    # print(f'Epoch [{epoch + 1}] Accuracy: {accuracy:.4f}, Validation Loss: {avg_loss:.4f}')
    return accuracy, avg_loss

In [ ]:
# Start training
train(5, model, mini_trains, mini_train_labels, mini_vals, mini_val_labels, device, loss_function, optimizer)

Iteration 0, Loss: 0.9485
Iteration 400, Loss: 0.7269
Iteration 800, Loss: 0.8439
Iteration 1200, Loss: 1.0232
Iteration 1600, Loss: 0.6940
Iteration 2000, Loss: 0.7749
Iteration 2400, Loss: 0.7437
Iteration 2800, Loss: 1.0277
Iteration 3200, Loss: 0.9852
Iteration 3600, Loss: 0.7045
Epoch [1] Training Loss: 0.7907, Training Accuracy: 0.6798
Epoch [1] Validation Loss: 0.7819, Validation Accuracy: 0.7039
Iteration 0, Loss: 0.7395
Iteration 400, Loss: 0.7359
Iteration 800, Loss: 0.8402
Iteration 1200, Loss: 1.0187
Iteration 1600, Loss: 0.6846
Iteration 2000, Loss: 0.7739
Iteration 2400, Loss: 0.7332
Iteration 2800, Loss: 1.0276
Iteration 3200, Loss: 0.9783
Iteration 3600, Loss: 0.6956
Epoch [2] Training Loss: 0.7877, Training Accuracy: 0.6812
Epoch [2] Validation Loss: 0.7794, Validation Accuracy: 0.7046
Iteration 0, Loss: 0.7453
Iteration 400, Loss: 0.7216
Iteration 800, Loss: 0.8386
Iteration 1200, Loss: 0.6500
Iteration 1600, Loss: 0.5327
Iteration 2000, Loss: 0.5020
Iteration 2400, L

In [ ]:
def preprocessing(reviews, patterns, replacements):
    lst = []
    for i in range(len(reviews)):
        review = reviews[i].lower()
        for pattern, replacement in zip(patterns, replacements):
            review = review.replace(pattern, replacement)
        lst.append(review)
    return lst

In [ ]:
macrumors_data = pd.read_csv('APIR_commemt_data.csv')
# macrumors_data = pd.read_csv('MacRumors - MacRumors.csv')

# 轉換日期格式
date = pd.to_datetime(macrumors_data['time'], format='%B %d, %Y %I:%M%p')
mr_after_data = macrumors_data[date > '2024-09-10']
mr_before_data = macrumors_data[date <= '2024-09-10']

print(mr_after_data[0])

mr_after_contents = mr_after_data['content'].astype(str).tolist()
mr_before_contents = mr_before_data['content'].astype(str).tolist()
print(len(mr_after_contents))
print(len(mr_before_contents))

20
10780


In [ ]:
before = preprocessing(mr_before_contents, patterns, replacements)
after = preprocessing(mr_after_contents, patterns, replacements)
labels = [0]*10800
reviews = after + before

In [ ]:
# num_train = 9319
# num_val = 36898
# longest_num_tokens = 200

num_train = 20
num_val = 10780
longest_num_tokens = 200



In [ ]:
def indexing_tokens():
    indices = {'<SOS>': 0, '<EOS>': 1, '<PAD>': 2, '<UNK>': 3}
    index = 4
    for i in range(num_train):
        review = reviews[i].split()
        for token in review:
            if token not in indices:
                indices[token] = index
                index += 1
    return indices

In [ ]:
def get_data(indices, longest_line_tokens, mode='train'):
    data = []
    Y = []
    if mode == 'train':
      for i in range(num_train):
        one_train_data = []
        y, review = labels[i], reviews[i]
        tokens = review.split()
        for token in tokens:
          one_train_data.append(indices[token])
          if len(one_train_data) == longest_line_tokens:
            break
        while len(one_train_data) < longest_line_tokens:
          one_train_data.append(indices['<PAD>'])
        one_train_data.insert(0, indices['<SOS>'])
        one_train_data.append(indices['<EOS>'])
        data.append(one_train_data)
        Y.append(y)
    else:
      for i in range(num_train, num_train+num_val):
        one_val_data = []
        y, review = labels[i], reviews[i]
        tokens = review.split()
        for token in tokens:
          if token in indices:
            one_val_data.append(indices[token])
          else:
            one_val_data.append(indices['<UNK>'])
          if len(one_val_data) == longest_line_tokens:
            break
        while len(one_val_data) < longest_line_tokens:
          one_val_data.append(indices['<PAD>'])
        one_val_data.insert(0, indices['<SOS>'])
        one_val_data.append(indices['<EOS>'])
        data.append(one_val_data)
        Y.append(y)
    return data, Y

In [ ]:
# Loading Training Data & Val Data
indices = indexing_tokens()
training_data, training_labels = get_data(indices, longest_num_tokens)
val_data, val_labels = get_data(indices, longest_num_tokens, mode='val')
print(len(training_data))
print(len(val_data))

20
10780


In [ ]:
# Create tensors of train & val
import numpy as np
train_tensor = torch.tensor(np.array(training_data))
train_labels_tensor = torch.tensor(np.array(training_labels))
val_tensor = torch.tensor(np.array(val_data))
val_labels_tensor = torch.tensor(np.array(val_labels))

print(len(train_tensor))
print(len(val_tensor))

20
10780


In [ ]:
mini_trains = DataLoader(train_tensor, batch_size=batch_size)
mini_train_labels = DataLoader(training_labels, batch_size=batch_size)

mini_vals = DataLoader(val_tensor, batch_size=batch_size)
mini_val_labels = DataLoader(val_labels, batch_size=batch_size)


print(mini_trains)

print(mini_vals)


In [ ]:
import pandas as pd

def save_predictions(predictions, comments, filename='APIR_commemt_data_IMDb_after.csv'):
    # Create a DataFrame from predictions and comments
    df = pd.DataFrame({
        'prediction': predictions,
        'comment': comments
    })

    # Save the DataFrame to a CSV file
    df.to_csv(filename, index=False)

# Example usage
# Assuming you have your predictions and comments
mr_predicted_results_after = predictor(model, 5, mini_trains, device, loss_function)
mr_predicted_results_before = predictor(model, 5, mini_vals, device, loss_function)

save_predictions(mr_predicted_results_after, mr_after_contents)
save_predictions(mr_predicted_results_before, mr_before_contents, filename='APIR_commemt_data_IMDb_before.csv')

In [ ]:
def predictor(model, epoch, mini_vals, device, loss_function):
    model.eval()
    with torch.no_grad():
        total_loss = 0.0
        all_predictions = []  # Initialize a list to store predictions

        for x in mini_vals:
            x = x.to(device)

            scores = model(x)
            predictions = scores.max(1)[1]  # Get the predicted class indices

            # Append predictions to the list
            all_predictions.extend(predictions.cpu().numpy().tolist())  # Convert to list and add to all_predictions

            # Calculate the loss for the current batch
            # Assuming you have the true labels available for loss calculation
            # loss = loss_function(scores, y)  # You can include this if you have labels for loss
            # total_loss += loss.item()

    # Optionally, if you still want to calculate the loss, you would need to adjust your inputs
    # return all_predictions if you do not want to calculate accuracy or loss
    return all_predictions  # Return only predictions



In [ ]:
import pandas as pd

# List of your CSV files
csv_files = ['MacRumors - MacRumors_IMDb_after.csv', 'MacRumors - MacRumors_IMDb_before.csv', 'Trustpilot_commemt_data_IMDb_after.csv', 'Trustpilot_commemt_data_IMDb_before.csv', 'APIR_commemt_data_IMDb_after.csv', 'APIR_commemt_data_IMDb_before.csv']

# Read and concatenate all CSV files into one DataFrame
merged_df = pd.concat([pd.read_csv(file) for file in csv_files])

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('IMDb_output.csv', index=False)

df_before = pd.read_csv('IMDb_output.csv')
print(df_before)
df = pd.read_csv('Updated_Data_with_Year_and_Month (1).csv')


df = df.drop(columns=['Content'])

# print(df['Time'])
# print(df['Month'])
# print(df['Year'])

df = pd.concat([df, df['Time']], ignore_index=True)
df = pd.concat([df, df['Month']], ignore_index=True)
df = pd.concat([df, df['Year']], ignore_index=True)
df = pd.concat([df, df_before['comment']], ignore_index=True)
df = pd.concat([df, df_before['prediction']], ignore_index=True)
print(df)


# # Strip any leading or trailing spaces from the 'Time' column
# df['Time'] = df['Time'].str.strip()

# # Convert the 'Time' column to datetime format, handling errors
# # df['Time'] = pd.to_datetime(df['Time'], errors='coerce')

# # Check for any rows where the conversion failed
# if df['Time'].isnull().any():
#     print("Warning: Some dates could not be converted. They will be set to NaT.")

# # Add 'Year' and 'Month' columns
# df['Year'] = df['Time'].dt.year
# df['Month'] = df['Time'].dt.month

# # Drop the 'Content' column
# print(df)


# Save the modified DataFrame to a new CSV file
df.to_csv('IMDb_output.csv', index=False)


       prediction                                            comment
0               0  What's the desirability of a (UK based) 2019 M...
1               2  You don’t mention any prices…..Personally I ha...
2               0  First off. If you do decide to get this, the8T...
3               0  Why do you say this❓You can easily add a myria...
4               0  It is possible, but has also caused a lot of i...
...           ...                                                ...
57109           0  \nPhilips Hue Play HDMI Sync Box will coordina...
57110           0  \nNBC 'Peacock' streaming service expected in ...
57111           0  \nApple car could be recharged by a vehicle-se...
57112           0  \nShould you upgrade to iPhone 11 Pro if you h...
57113           0  \nApple argues that $14.4B EU tax order 'defie...

[57114 rows x 2 columns]
              Time    Year  Month    0
0       2024-09-11  2024.0    9.0  NaN
1       2024-09-11  2024.0    9.0  NaN
2       2024-09-11  2024.0   

In [ ]:
import pandas as pd

# List of your CSV files
csv_files = [
    'MacRumors - MacRumors_IMDb_after.csv',
    'MacRumors - MacRumors_IMDb_before.csv',
    'Trustpilot_commemt_data_IMDb_after.csv',
    'Trustpilot_commemt_data_IMDb_before.csv',
    'APIR_commemt_data_IMDb_after.csv',
    'APIR_commemt_data_IMDb_before.csv'
]
print(merged_df.shape)
print(new_data.shape)


# Read and concatenate all CSV files into one DataFrame
merged_df = pd.concat([pd.read_csv(file) for file in csv_files])

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('IMDb_output.csv', index=False)

# Read the combined CSV and the updated CSV with Year and Month
df_before = pd.read_csv('IMDb_output.csv')
df = pd.read_csv('Updated_Data_with_Year_and_Month (1).csv')

# Drop the 'Content' column from the second DataFrame
df = df.drop(columns=['Content'])

# Create a new DataFrame with the necessary columns
new_data = pd.DataFrame({
    'Time': df['Time'],
    'Month': df['Month'],
    'Year': df['Year'],
})

print(new_data)
# Concatenate new_data with the existing merged_df
final_df = pd.concat([merged_df, new_data], ignore_index=True)
print(final_df)
# Save the modified DataFrame to a new CSV file
final_df.to_csv('final_IMDb_output.csv', index=False)

# Print the final DataFrame
print(final_df)


(57114, 2)
(57114, 3)
                      Time  Month  Year
0               2024-09-11      9  2024
1               2024-09-11      9  2024
2               2024-09-11      9  2024
3               2024-09-11      9  2024
4               2024-09-11      9  2024
...                    ...    ...   ...
57109  2019-09-19 12:35:00      9  2019
57110  2019-09-19 12:35:00      9  2019
57111  2019-09-19 12:35:00      9  2019
57112  2019-09-19 12:35:00      9  2019
57113  2019-09-19 12:35:00      9  2019

[57114 rows x 3 columns]
        prediction                                            comment  \
0              0.0  What's the desirability of a (UK based) 2019 M...   
1              2.0  You don’t mention any prices…..Personally I ha...   
2              0.0  First off. If you do decide to get this, the8T...   
3              0.0  Why do you say this❓You can easily add a myria...   
4              0.0  It is possible, but has also caused a lot of i...   
...            ...                

In [ ]:
import pandas as pd

# List of your CSV files
csv_files = [
    'MacRumors - MacRumors_IMDb_after.csv',
    'MacRumors - MacRumors_IMDb_before.csv',
    'Trustpilot_commemt_data_IMDb_after.csv',
    'Trustpilot_commemt_data_IMDb_before.csv',
    'APIR_commemt_data_IMDb_after.csv',
    'APIR_commemt_data_IMDb_before.csv'
]

# Read and concatenate all CSV files into one DataFrame
merged_df = pd.concat([pd.read_csv(file) for file in csv_files])

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('IMDb_output.csv', index=False)

# Read the combined CSV and the updated CSV with Year and Month
df_before = pd.read_csv('IMDb_output.csv')
df = pd.read_csv('Updated_Data_with_Year_and_Month (1).csv')

# Drop the 'Content' column from the second DataFrame if it exists
if 'Content' in df.columns:
    df = df.drop(columns=['Content'])

# Create a new DataFrame with the necessary columns
new_data = pd.DataFrame({
    'Time': df['Time'],
    'Month': df['Month'],
    'Year': df['Year'],
})

# Ensure the number of rows matches
if merged_df.shape[0] != new_data.shape[0]:
    print("Warning: The number of rows does not match.")
else:
    # Reset indices to avoid InvalidIndexError
    merged_df.reset_index(drop=True, inplace=True)
    new_data.reset_index(drop=True, inplace=True)

    # Concatenate merged_df and new_data side by side
    final_df = pd.concat([merged_df, new_data], axis=1)

    # Rename columns if necessary to ensure correct final structure
    final_df.columns = ['prediction', 'comment', 'Time', 'Month', 'Year']

    # Check the final DataFrame shape
    print("Final DF shape:", final_df.shape)

    # Save the modified DataFrame to a new CSV file
    final_df.to_csv('final_IMDb_output.csv', index=False)

    # Print the final DataFrame
    print(final_df)


Final DF shape: (57114, 5)
       prediction                                            comment  \
0               0  What's the desirability of a (UK based) 2019 M...   
1               2  You don’t mention any prices…..Personally I ha...   
2               0  First off. If you do decide to get this, the8T...   
3               0  Why do you say this❓You can easily add a myria...   
4               0  It is possible, but has also caused a lot of i...   
...           ...                                                ...   
57109           0  \nPhilips Hue Play HDMI Sync Box will coordina...   
57110           0  \nNBC 'Peacock' streaming service expected in ...   
57111           0  \nApple car could be recharged by a vehicle-se...   
57112           0  \nShould you upgrade to iPhone 11 Pro if you h...   
57113           0  \nApple argues that $14.4B EU tax order 'defie...   

                      Time  Month  Year  
0               2024-09-11      9  2024  
1               2024-09-

In [ ]:
# 品牌聲譽評分設定
def calculate_score(predicted_result):
    nominator = 0
    denominator = 0

    denominator = len(predicted_result)
    # positive_count = sum(1 for result in predicted_result if result == 2)
    # negative_count = sum(1 for result in predicted_result if result == 0)
    counts = predicted_result.value_counts()
    positive_count = counts.get(2)  # Default to 0 if 2 is not found
    negative_count = counts.get(0)  # Default to 0 if 0 is not found
    nominator = positive_count - negative_count

    return (nominator / denominator) * 100

In [ ]:
import pandas as pd

mr_predicted_results_before = pd.read_csv('MacRumors - MacRumors_IMDb_before.csv')
mr_predicted_results_after = pd.read_csv('MacRumors - MacRumors_IMDb_after.csv')

mr_before_score = calculate_score(mr_predicted_results_before['prediction'])
mr_after_score = calculate_score(mr_predicted_results_after['prediction'])

tp_predicted_results_before = pd.read_csv('Trustpilot_commemt_data_IMDb_before.csv')
tp_predicted_results_after = pd.read_csv('Trustpilot_commemt_data_IMDb_after.csv')

tp_before_score = calculate_score(tp_predicted_results_before['prediction'])
tp_after_score = calculate_score(tp_predicted_results_after['prediction'])

ap_predicted_results_before = pd.read_csv('APIR_commemt_data_IMDb_before.csv')
ap_predicted_results_after = pd.read_csv('APIR_commemt_data_IMDb_after.csv')

ap_before_score = calculate_score(ap_predicted_results_before['prediction'])
ap_after_score = calculate_score(ap_predicted_results_after['prediction'])

# Read CSV file
total_predicted_results = pd.read_csv('final_IMDb_output.csv')

total_predicted_results['Time'] = pd.to_datetime(total_predicted_results['Time'], errors='coerce')  # errors='coerce' 將無法解析的值轉為 NaT

# Filter results by time
before_df = total_predicted_results[(total_predicted_results['Time'] >= '2024-08-01') & (total_predicted_results['Time'] <= '2024-09-10')]
after_df = total_predicted_results[total_predicted_results['Time'] > '2024-09-10']


# 然後對篩選後的數據計算分數
total_before_score = calculate_score(before_df['prediction'])
total_after_score = calculate_score(after_df['prediction'])

print(mr_before_score)
print(mr_after_score)

print(tp_before_score)
print(tp_after_score)

print(ap_before_score)
print(ap_after_score)

print(total_before_score)
print(total_after_score)

-7.734836576508212
-2.3285760274707585
-91.17647058823529
24.137931034482758
-91.02040816326532
10.0
-11.357340720221606
-2.3285760274707585
